In [1]:
if __name__ == "__main__":
    from statsmodels.stats.power import FTestAnovaPower

    # --- Our "Ingredients" ---
    effect_size = None   # Cohen's f for effect size from previous analysis
    alpha = 0.05        # Our significance level
    power = 0.80        # Our desired power (80%)
    k_groups = 3     # Number of groups
    nobs = 26
    # --- The Power Calculation ---
    # We use the solve_power function. We provide the knowns and set
    # the value we want to find (nobs, number of observations) to None.
    power_analysis = FTestAnovaPower()
    result = power_analysis.solve_power(
        effect_size=effect_size, #solve for effect size
        alpha=alpha,
        power=0.8,
        k_groups=k_groups,
        nobs=nobs
        
    )

    print(f"{k_groups}-way ANOVA result: {result:.3f}")

    # sample_size
    # # --- Print the Result ---
    # print(f"Required sample size per group: {round(sample_size)}")
    # print(f"Total sample size: {round(sample_size) * k_groups}")

3-way ANOVA result: 0.651


In [2]:
if __name__ == "__main__":
    from statsmodels.stats.power import TTestIndPower

    # --- Our "Ingredients" ---
    effect_size = None  # Cohen's d for effect <= get the effect size from previous analysis
    alpha = 0.05        # Our significance level
    power = 0.80        # Our desired power (80%)
    nobs = 10

    # --- The Power Calculation ---
    # We use the solve_power function for an independent t-test.
    # We are solving for the number of observations per group (nobs1).
    # ratio=1.0 means we want equal sample sizes in both groups.
    power_analysis = TTestIndPower()
    result = power_analysis.solve_power(
        effect_size=effect_size, # solve for effect size
        alpha=alpha,
        power=power,
        ratio=1.0,  # For equal group sizes
        alternative="two-sided",
        nobs1=nobs
    )

    print(f"T-Test result: {result:.3f}")

    # # --- Print the Result ---
    # print(f"Required sample size per group: {round(sample_size)}")
    # print(f"Total sample size: {round(sample_size) * 2}")

T-Test result: 1.325


In [3]:
if __name__ == "__main__":
    import statsmodels.stats.power as smp
    import pandas as pd
    import numpy as np  

    # Parameters for the power analysis
    nobs = 87
    alpha = 0.05
    power = 0.80
    k_regressors = 2

    # Degrees of freedom for the F-test
    df_num = k_regressors
    df_denom = nobs - k_regressors - 1

    # Create an instance of the FTestPower class
    power_analysis = smp.FTestPower()

    # Perform the power analysis to find the non-centrality parameter.
    # The `solve_power` method returns this value when `effect_size` is None.
    cohen_f = power_analysis.solve_power(
        effect_size=None,
        alpha=alpha,
        power=power,
        df_num=df_num,
        df_denom=df_denom,
    )

    # Cohen's f^2 is calculated by dividing the non-centrality parameter by the number of observations.
    # make this cohen's f 
    cohen_f2 = cohen_f**2 
    r_squared = cohen_f2 / (1 + cohen_f2)

    # Print the result
    print(f"The required Cohen's f^2 effect size is: {cohen_f2:.4f}")
    print(f"The required Cohen's f effect size is: {cohen_f:.4f}")
    print(f"This corresponds to an R-squared of: {r_squared:.4f}")

The required Cohen's f^2 effect size is: 29.3476
The required Cohen's f effect size is: 5.4173
This corresponds to an R-squared of: 0.9670


In [4]:
if __name__ == "__main__":
    import numpy as np
    import pandas as pd
    from statsmodels.stats.power import FTestPower
    from statsmodels.regression.linear_model import OLS
    import statsmodels.api as sm

    # Parameters for your regression
    n_obs = 87  # number of observations
    k = 2       # number of regressors (excluding intercept)
    df_num = k  # numerator degrees of freedom
    df_den = n_obs - k - 1  # denominator degrees of freedom (84)

    print(f"Regression setup:")
    print(f"- Observations: {n_obs}")
    print(f"- Regressors: {k}")
    print(f"- df numerator: {df_num}")
    print(f"- df denominator: {df_den}")
    print("-" * 50)

    # Method 1: Using FTestPower for power analysis
    # This helps determine what effect sizes are detectable with your sample
    power_test = FTestPower()

    # Calculate minimum detectable effect size (f-squared) for 80% power
    alpha = 0.05
    power = 0.80

    min_f_squared = power_test.solve_power(effect_size=None, 
                                        nobs=n_obs, 
                                        alpha=alpha, 
                                        power=power, 
                                        df_num=df_num,
                                            df_denom=df_denom)

    print(f"Power Analysis Results:")
    print(f"Minimum detectable f² for 80% power: {min_f_squared:.4f}")

    # Convert f-squared to R-squared
    min_r_squared = min_f_squared / (1 + min_f_squared)
    print(f"Corresponding minimum R²: {min_r_squared:.4f}")

Regression setup:
- Observations: 87
- Regressors: 2
- df numerator: 2
- df denominator: 84
--------------------------------------------------
Power Analysis Results:
Minimum detectable f² for 80% power: 5.4173
Corresponding minimum R²: 0.8442


/home/vscode/.local/lib/python3.12/site-packages/statsmodels/stats/power.py:1126: UserWarning: nobs is not used
  warnings.warn("nobs is not used")


In [6]:
import numpy as np
from scipy import stats
from scipy.optimize import fsolve

def calculate_target_cohens_f(nobs, n_regressors, power=0.80, alpha=0.05):
    """
    Calculate target Cohen's f effect size for multivariate linear regression
    to achieve specified statistical power.
    
    Parameters:
    -----------
    nobs : int
        Number of observations in the dataset
    n_regressors : int  
        Number of independent variables (predictors) in the regression
    power : float, default=0.80
        Desired statistical power (probability of detecting true effect)
    alpha : float, default=0.05
        Significance level (Type I error rate)
    
    Returns:
    --------
    dict
        Dictionary containing:
        - 'cohens_f': Cohen's f effect size
        - 'cohens_f_squared': Cohen's f-squared effect size  
        - 'r_squared': R-squared (proportion of variance explained)
        
    Notes:
    ------
    - Uses F-test for overall regression significance
    - Based on non-central F-distribution for power calculations
    - Relationships: f² = R²/(1-R²) and f = √(f²)
    
    Example:
    --------
    >>> result = calculate_target_cohens_f(nobs=87, n_regressors=2)
    >>> print(f"Target R²: {result['r_squared']:.4f}")
    >>> print(f"Target Cohen's f: {result['cohens_f']:.4f}")
    """
    
    # Input validation
    if nobs <= n_regressors + 1:
        raise ValueError(f"Sample size ({nobs}) must be greater than number of regressors + 1 ({n_regressors + 1})")
    
    if not (0 < power < 1):
        raise ValueError("Power must be between 0 and 1")
        
    if not (0 < alpha < 1):
        raise ValueError("Alpha must be between 0 and 1")
    
    # Degrees of freedom
    df_numerator = n_regressors  # df for regression
    df_denominator = nobs - n_regressors - 1  # df for error
    
    def calculate_power_from_f_squared(f_squared):
        """
        Calculate statistical power given Cohen's f-squared
        """
        # Critical F-value for given alpha
        f_critical = stats.f.ppf(1 - alpha, df_numerator, df_denominator)
        
        # Non-centrality parameter
        ncp = nobs * f_squared
        
        # Power using non-central F-distribution
        power_calculated = 1 - stats.ncf.cdf(f_critical, df_numerator, df_denominator, ncp)
        
        return power_calculated
    
    def power_equation(f_squared):
        """
        Equation to solve: calculated_power - target_power = 0
        """
        return calculate_power_from_f_squared(f_squared) - power
    
    # Find Cohen's f-squared that achieves target power
    try:
        # Initial guess based on typical medium effect size
        initial_guess = 0.15
        
        # Solve for f-squared
        cohens_f_squared = fsolve(power_equation, initial_guess)[0]
        
        # Ensure positive result
        if cohens_f_squared < 0:
            cohens_f_squared = fsolve(power_equation, 0.05)[0]
            
    except Exception as e:
        raise RuntimeError(f"Could not solve for target effect size: {e}")
    
    # Calculate derived effect size measures
    cohens_f = np.sqrt(cohens_f_squared)
    r_squared = cohens_f_squared / (1 + cohens_f_squared)
    
    # Verify the solution
    actual_power = calculate_power_from_f_squared(cohens_f_squared)
    
    # Warning if solution is not accurate
    if abs(actual_power - power) > 0.01:
        print(f"Warning: Achieved power ({actual_power:.4f}) differs from target ({power:.4f})")
    
    return {
        'cohens_f': cohens_f,
        'cohens_f_squared': cohens_f_squared,
        'r_squared': r_squared
    }


# Example usage and testing
if __name__ == "__main__":
    print("=== TESTING calculate_target_cohens_f FUNCTION ===\n")
    
    # Test with your original example
    print("Test 1: Your original study (N=87, k=2)")
    result1 = calculate_target_cohens_f(nobs=87, n_regressors=2)
    print(f"Target Cohen's f: {result1['cohens_f']:.4f}")
    print(f"Target Cohen's f²: {result1['cohens_f_squared']:.4f}")  
    print(f"Target R²: {result1['r_squared']:.4f} ({result1['r_squared']*100:.2f}% variance)")
    
    # Verify with your actual results
    your_r_squared = 0.151
    your_f_squared = your_r_squared / (1 - your_r_squared)
    print(f"\nYour actual R²: {your_r_squared:.4f}")
    print(f"Your actual f²: {your_f_squared:.4f}")
    print(f"Conclusion: {'✓ Well-powered' if your_f_squared > result1['cohens_f_squared'] else '✗ Underpowered'}")
    
    print(f"\n" + "="*50)
    
    # Test with different sample sizes
    print("Test 2: Different sample sizes (k=2 regressors)")
    sample_sizes = [30, 50, 75, 100, 150, 200]
    
    for n in sample_sizes:
        if n > 3:  # Ensure valid df
            result = calculate_target_cohens_f(nobs=n, n_regressors=2)
            print(f"N={n:3d}: Target R² = {result['r_squared']:.4f}, f² = {result['cohens_f_squared']:.4f}")
    
    print(f"\n" + "="*50)
    
    # Test with different numbers of regressors
    print("Test 3: Different numbers of regressors (N=87)")
    regressors = [1, 2, 3, 5, 8, 10]
    
    for k in regressors:
        if 87 > k + 1:  # Ensure valid df
            result = calculate_target_cohens_f(nobs=87, n_regressors=k)
            print(f"k={k:2d}: Target R² = {result['r_squared']:.4f}, f² = {result['cohens_f_squared']:.4f}")
    
    print(f"\n" + "="*50)
    
    # Test with different power levels
    print("Test 4: Different power levels (N=87, k=2)")
    power_levels = [0.70, 0.80, 0.90, 0.95]
    
    for power_target in power_levels:
        result = calculate_target_cohens_f(nobs=87, n_regressors=2, power=power_target)
        print(f"Power={power_target:.2f}: Target R² = {result['r_squared']:.4f}, f² = {result['cohens_f_squared']:.4f}")
    
    print(f"\n=== FUNCTION TESTING COMPLETE ===")

=== TESTING calculate_target_cohens_f FUNCTION ===

Test 1: Your original study (N=87, k=2)
Target Cohen's f: 0.3388
Target Cohen's f²: 0.1148
Target R²: 0.1030 (10.30% variance)

Your actual R²: 0.1510
Your actual f²: 0.1779
Conclusion: ✓ Well-powered

Test 2: Different sample sizes (k=2 regressors)
N= 30: Target R² = 0.2645, f² = 0.3596
N= 50: Target R² = 0.1705, f² = 0.2055
N= 75: Target R² = 0.1181, f² = 0.1340
N=100: Target R² = 0.0904, f² = 0.0994
N=150: Target R² = 0.1304, f² = 0.1500
N=200: Target R² = 0.1304, f² = 0.1500

Test 3: Different numbers of regressors (N=87)
k= 1: Target R² = 0.0845, f² = 0.0923
k= 2: Target R² = 0.1030, f² = 0.1148
k= 3: Target R² = 0.1161, f² = 0.1314
k= 5: Target R² = 0.1363, f² = 0.1578
k= 8: Target R² = 0.1599, f² = 0.1903
k=10: Target R² = 0.1734, f² = 0.2097

Test 4: Different power levels (N=87, k=2)
Power=0.70: Target R² = 0.0840, f² = 0.0918
Power=0.80: Target R² = 0.1030, f² = 0.1148
Power=0.90: Target R² = 0.1310, f² = 0.1508
Power=0.95: 

/tmp/ipykernel_18255/561705490.py:83: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  cohens_f_squared = fsolve(power_equation, initial_guess)[0]
